In [3]:
import sqlite3
from Character import Character
from Move import Move
from Battle import Battle
from Ability import Ability
from db import DatabaseManager

In [6]:
def create_ability_from_db(rec):
    return Ability(
        rec["name"],
        rec["stat_target"],
        rec["boost_value"],
        rec["penalty_value"],
        rec["active_duration"],
        rec["penalty_duration"]
    )

def main():
    MAX_HEALTH = 100
    db = DatabaseManager()

    
    db.add_item("Punch")
    db.add_item("Steal")
    db.add_item("Super Speed",    "speed",    30, 20, active_duration=4, penalty_duration=2)
    db.add_item("Super Strength", "strength", 30, 20, active_duration=3, penalty_duration=2)
    

    while True:
        
        with sqlite3.connect(db.db_path) as conn:
            conn.execute("UPDATE items SET is_borrowed = 0")

        #Create player
        print("Create your character:")
        name = input("Name? ")
        age  = int(input("Age? "))
        gen  = int(input("Gender (1=male, 2=female)? "))
        gender_str = "male" if gen == 1 else "female"

        user_id = db.add_user(
            name,
            age,
            gender_str,
            health=MAX_HEALTH,
            speed=50,
            strength=50,
            durability=50
        )

        player = Character(
            name, age, gen,
            health=MAX_HEALTH,
            speed=50,
            strength=50,
            durability=50,
            ability=None,
            is_enemy=False,
            db=db,
            user_id=user_id
        )

        #Love interest
        love_name = input("Love interest name? ")
        love_gen  = int(input("Gender (1=male, 2=female)? "))
        love = Character(love_name, age, love_gen)

        #Define enemies based on player gender
        male_defs = {
            "Stanley": ("Super Speed",    50, 100, 50, 50),
            "Robert": ("Super Strength", 100,  70,100, 50),
        }
        female_defs = {
            "JY":  ("Super Speed",    50, 100, 50, 50),
            "Amy": ("Super Strength", 50,  70,100, 50),
        }
        defs = male_defs if player.gender == "male" else female_defs
        enemy_gender_code = 1 if player.gender == "male" else 2

        enemies = []
        for ename, (abil_name, hp, sp, st, du) in defs.items():
            rec = db.get_item_by_name(abil_name)
            abil = create_ability_from_db(rec)
            e = Character(
                ename, age, enemy_gender_code,
                health=hp, speed=sp, strength=st, durability=du,
                ability=abil, is_enemy=True,
                db=db, user_id=None
            )
            enemies.append(e)

        #Show initial state
        print("\nYour character:")
        print(" ", player)
        print("\nEnemies:")
        for e in enemies:
            print(" ", e)

        #First Battle
        print("\nFirst Battle")
        battle1 = Battle(player, enemies[0], db=db, user_id=user_id)
        battle1.start_battle()

        #Health restoration prompt
        if player.health > 0:
            choice = input(
                "You won the first battle! Restore your health to full before next battle?\n"
                "Type 1 for Yes, 2 for No: "
            ).strip()
            if choice == "1":
                player.health = MAX_HEALTH
                print(f"Health restored to {player.health}.\n")

        #Second Battle
        if player.health > 0:
            print("Next Battle: Super Strength Enemy")
            enemy_ss = next(e for e in enemies if e.ability and e.ability.name == "Super Strength")
            battle2 = Battle(player, enemy_ss, db=db, user_id=user_id)
            battle2.start_battle()

        #Final reunion sequence
        if player.health > 0:
            print(f"\nYou have defeated all the bosses. Now, will you go to reunite with {love.name}?")
            choice = input("1 = Yes, 2 = No: ").strip()
            if choice == "1":
                print(f"\nYou stand before {love.name}. How will you demonstrate your love and strength?")
                print("1. Demonstrate Super Strength")
                print("2. Do nothing")
                action = input("Choose 1 or 2: ").strip()
                if action == "1":
                    if player.ability and player.ability.name == "Super Strength":
                        print(f"\nYou flex your mighty muscles and lift a massive boulder to protect {love.name}.")
                        print(f"{love.name} is awestruck and embraces you. You win! The end.")
                    else:
                        print(f"\nYou try to impress with Super Strength, but without the power, it’s all for naught.")
                        print(f"{love.name} looks disappointed and walks away. Game Over!")
                else:
                    print(f"\nYou hesitate and do nothing. {love.name} feels ignored and leaves. Game Over!")
            else:
                print(f"\nYou decide not to approach {love.name}. {love.name} is hurt and walks away. Game Over!")
        else:
            print("\nGame Over!")

        # 9) Restart?
        restart = input("\nRestart game? (Type 1 for Yes): ").strip()
        if restart != "1":
            break

if __name__ == "__main__":
    main()


Create your character:

Your character:
  Name: Aabid  Health: 100  Speed: 50  Strength: 50  Durability: 50  Ability: None

Enemies:
  Name: Stanley  Health: 50  Speed: 100  Strength: 50  Durability: 50  Ability: Super Speed
  Name: Robert  Health: 100  Speed: 70  Strength: 100  Durability: 50  Ability: Super Strength

First Battle
Battle begins: Aabid vs. Stanley
Stanley activates Super Speed
Super Speed has been activated
Your moves:
1. Punch
2. Steal
Aabid uses Punch dealing 25 damage.
Stanley has 25 health left.
Stanley attacks with Punch dealing 25 damage.
Aabid has 75 health left.
Your moves:
1. Punch
2. Steal
Stanley has dodged the attack with Super Speed!
Stanley attacks with Punch dealing 25 damage.
Aabid has 50 health left.
Your moves:
1. Punch
2. Steal
Aabid uses Punch dealing 25 damage.
Stanley has 0 health left.
Aabid has won!
Aabid steals Stanley's ability: Super Speed
Ability stolen: Super Speed
Health restored to 100.

Next Battle: Super Strength Enemy
Battle begins: Aa

ValueError: invalid literal for int() with base 10: ''